In [1]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from numba import cuda

from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Conv1D, GlobalMaxPooling1D,LSTM

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import keras.backend as K

#dtype='float16'
#K.set_floatx(dtype)

# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
#K.set_epsilon(1e-3) 

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12256108747207451447
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4557766656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12136950183195905415
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
def free_gpu_mem():
    cuda.select_device(0)
    cuda.close()

In [6]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [7]:
word_vec_size=300

In [8]:
def w2v_per_token_wiki(word_vec_size):
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))

    embeddings_index,songs=utils.get_embedding_dict(songs,'glove-wiki-gigaword-300')
    print(len(embeddings_index.keys()))
    max_words=len(embeddings_index.keys())
    X,y=np.array(songs),np.array(labels)

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)

    length = []
    for x in X:
        length.append(len(x))

    X = pad_sequences(sequences, maxlen=max(length)+2)

    embedding_matrix = np.zeros((max_words, word_vec_size))
    for word, i in tokenizer.word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.5)
    
    
    return x_train,x_test,y_train,y_test,embedding_matrix,max(length)+2,max_words

In [9]:
x_train,x_test,y_train,y_test,embedding_matrix,max_sent_len,max_words=w2v_per_token_wiki(word_vec_size)

19704


In [10]:
def dummy(x_train,x_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(x_train, y_train)
    y_pred=dummy.predict(x_test)
    return f1_score(y_test,y_pred)

In [11]:
print(dummy(x_train,x_test,y_train,y_test))

0.4614871623874625


In [12]:
def dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [13]:
print(dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 9s - loss: 0.8708 - acc: 0.5639 - val_loss: 0.6110 - val_acc: 0.6304
Epoch 2/100
 - 4s - loss: 0.4950 - acc: 0.7295 - val_loss: 0.5651 - val_acc: 0.6569
Epoch 3/100
 - 4s - loss: 0.2878 - acc: 0.8828 - val_loss: 0.6070 - val_acc: 0.6698
Epoch 4/100
 - 5s - loss: 0.1550 - acc: 0.9474 - val_loss: 0.6556 - val_acc: 0.6550
Epoch 5/100
 - 4s - loss: 0.1028 - acc: 0.9717 - val_loss: 0.7179 - val_acc: 0.6638
Epoch 6/100
 - 5s - loss: 0.0749 - acc: 0.9739 - val_loss: 0.7918 - val_acc: 0.6608
Epoch 7/100
 - 5s - loss: 0.0611 - acc: 0.9781 - val_loss: 0.7717 - val_acc: 0.6517
Epoch 8/100
 - 5s - loss: 0.0705 - acc: 0.9790 - val_loss: 0.7955 - val_acc: 0.6535
Epoch 9/100
 - 5s - loss: 0.0655 - acc: 0.983

Epoch 86/100
 - 5s - loss: 0.0954 - acc: 0.9814 - val_loss: 1.7999 - val_acc: 0.6520
Epoch 87/100
 - 5s - loss: 0.0691 - acc: 0.9832 - val_loss: 1.6567 - val_acc: 0.6553
Epoch 88/100
 - 5s - loss: 0.0630 - acc: 0.9859 - val_loss: 1.8425 - val_acc: 0.6547
Epoch 89/100
 - 5s - loss: 0.0623 - acc: 0.9850 - val_loss: 1.8335 - val_acc: 0.6496
Epoch 90/100
 - 4s - loss: 0.0659 - acc: 0.9853 - val_loss: 1.9053 - val_acc: 0.6505
Epoch 91/100
 - 5s - loss: 0.0568 - acc: 0.9841 - val_loss: 1.6869 - val_acc: 0.6490
Epoch 92/100
 - 5s - loss: 0.0559 - acc: 0.9865 - val_loss: 2.0890 - val_acc: 0.6496
Epoch 93/100
 - 5s - loss: 0.0770 - acc: 0.9838 - val_loss: 1.9892 - val_acc: 0.6517
Epoch 94/100
 - 5s - loss: 0.0770 - acc: 0.9856 - val_loss: 1.8857 - val_acc: 0.6593
Epoch 95/100
 - 5s - loss: 0.0567 - acc: 0.9877 - val_loss: 2.1272 - val_acc: 0.6499
Epoch 96/100
 - 5s - loss: 0.0637 - acc: 0.9856 - val_loss: 2.1279 - val_acc: 0.6523
Epoch 97/100
 - 5s - loss: 0.0611 - acc: 0.9847 - val_loss: 2.106

In [14]:
def dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [15]:
print(dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 9s - loss: 0.7542 - acc: 0.5906 - val_loss: 0.5948 - val_acc: 0.6532
Epoch 2/100
 - 5s - loss: 0.4610 - acc: 0.7797 - val_loss: 0.5688 - val_acc: 0.6626
Epoch 3/100
 - 5s - loss: 0.2600 - acc: 0.9062 - val_loss: 0.6407 - val_acc: 0.6529
Epoch 4/100
 - 5s - loss: 0.1392 - acc: 0.9639 - val_loss: 0.6778 - val_acc: 0.6599
Epoch 5/100
 - 5s - loss: 0.0994 - acc: 0.9732 - val_loss: 0.7637 - val_acc: 0.6644
Epoch 6/100
 - 5s - loss: 0.0872 - acc: 0.9781 - val_loss: 0.7877 - val_acc: 0.6623
Epoch 7/100
 - 5s - loss: 0.0758 - acc: 0.9748 - val_loss: 0.8633 - val_acc: 0.6638
Epoch 8/100
 - 5s - loss: 0.0604 - acc: 0.9799 - val_loss: 0.9589 - val_acc: 0.6572
Epoch 9/100
 - 5s - loss: 0.0514 - acc: 0.9820 - val_loss: 0.8920 - val_acc: 0.6686
Epoch 10/100
 - 5s - loss: 0.0509 - acc: 0.9802 - val_loss: 0.9176 - val_acc: 0.6614
Epoch 11/100
 - 5s - loss: 0.0486 - acc: 0.9844 - val_loss: 1.0262 - val_acc: 0.6713
Epoch 12/100
 - 5s - loss:

Epoch 97/100
 - 5s - loss: 0.0379 - acc: 0.9871 - val_loss: 2.3738 - val_acc: 0.6689
Epoch 98/100
 - 5s - loss: 0.0374 - acc: 0.9889 - val_loss: 2.4200 - val_acc: 0.6662
Epoch 99/100
 - 5s - loss: 0.0435 - acc: 0.9877 - val_loss: 2.2940 - val_acc: 0.6686
Epoch 100/100
 - 5s - loss: 0.0385 - acc: 0.9865 - val_loss: 2.2066 - val_acc: 0.6683
0.6198347107438017


In [16]:
def cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=16, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [17]:
print(cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 0.6921 - acc: 0.5618 - val_loss: 0.6454 - val_acc: 0.6289
Epoch 2/100
 - 2s - loss: 0.5849 - acc: 0.6784 - val_loss: 0.6897 - val_acc: 0.5892
Epoch 3/100
 - 2s - loss: 0.4411 - acc: 0.7953 - val_loss: 0.6830 - val_acc: 0.6436
Epoch 4/100
 - 2s - loss: 0.2818 - acc: 0.8954 - val_loss: 0.6646 - val_acc: 0.6529
Epoch 5/100
 - 2s - loss: 0.1714 - acc: 0.9528 - val_loss: 0.7944 - val_acc: 0.6590
Epoch 6/100
 - 2s - loss: 0.1296 - acc: 0.9618 - val_loss: 0.9976 - val_acc: 0.6599
Epoch 7/100
 - 2s - loss: 0.1140 - acc: 0.9729 - val_loss: 0.9144 - val_acc: 0.6614
Epoch 8/100
 - 2s - loss: 0.0965 - acc: 0.9754 - val_loss: 0.8515 - val_acc: 0.6547
Epoch 9/100
 - 2s - loss: 0.0824 - acc: 0.9787 - val_loss: 0.9453 - val_acc: 0.6614
Epoch 10/100
 - 2s - loss: 0.0711 - acc: 0.9823 - val_loss: 0.8943 - val_acc: 0.6463
Epoch 11/100
 - 2s - loss: 0.0599

Epoch 94/100
 - 2s - loss: 0.0240 - acc: 0.9859 - val_loss: 2.4611 - val_acc: 0.6481
Epoch 95/100
 - 2s - loss: 0.0228 - acc: 0.9865 - val_loss: 2.6038 - val_acc: 0.6526
Epoch 96/100
 - 2s - loss: 0.0240 - acc: 0.9865 - val_loss: 2.5769 - val_acc: 0.6593
Epoch 97/100
 - 2s - loss: 0.0238 - acc: 0.9871 - val_loss: 2.2343 - val_acc: 0.6584
Epoch 98/100
 - 2s - loss: 0.0231 - acc: 0.9865 - val_loss: 2.7345 - val_acc: 0.6575
Epoch 99/100
 - 2s - loss: 0.0218 - acc: 0.9862 - val_loss: 2.7562 - val_acc: 0.6484
Epoch 100/100
 - 2s - loss: 0.0216 - acc: 0.9871 - val_loss: 2.7949 - val_acc: 0.6469
0.5907349355625218


In [18]:
def cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=16, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [19]:
print(cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 5s - loss: 0.6747 - acc: 0.5837 - val_loss: 0.6195 - val_acc: 0.6241
Epoch 2/100
 - 4s - loss: 0.4985 - acc: 0.7586 - val_loss: 0.5639 - val_acc: 0.6635
Epoch 3/100
 - 4s - loss: 0.2309 - acc: 0.9200 - val_loss: 0.6444 - val_acc: 0.6734
Epoch 4/100
 - 4s - loss: 0.1208 - acc: 0.9705 - val_loss: 0.7132 - val_acc: 0.6608
Epoch 5/100
 - 4s - loss: 0.0875 - acc: 0.9763 - val_loss: 0.7293 - val_acc: 0.6698
Epoch 6/100
 - 4s - loss: 0.0748 - acc: 0.9796 - val_loss: 0.6807 - val_acc: 0.6635
Epoch 7/100
 - 4s - loss: 0.0552 - acc: 0.9826 - val_loss: 0.7584 - val_acc: 0.6665
Epoch 8/100
 - 4s - loss: 0.0411 - acc: 0.9856 - val_loss: 0.7520 - val_acc: 0.6662
Epoch 9/100
 - 4s - loss: 0.0354 - acc: 0.9817 - val_loss: 0.8294 - val_acc: 0.6632
Epoch 10/100
 - 4s - loss: 0.0316 - acc: 0.9841 - val_loss: 0.8717 - val_acc: 0.6665
Epoch 11/100
 - 4s - loss: 0.0274 - acc: 0.9862 - val_loss: 0.8614 - val_acc: 0.6656
Epoch 12/100
 - 4s - loss:

Epoch 97/100
 - 4s - loss: 0.0213 - acc: 0.9874 - val_loss: 2.4813 - val_acc: 0.6644
Epoch 98/100
 - 4s - loss: 0.0212 - acc: 0.9871 - val_loss: 2.5159 - val_acc: 0.6638
Epoch 99/100
 - 4s - loss: 0.0223 - acc: 0.9880 - val_loss: 2.5405 - val_acc: 0.6656
Epoch 100/100
 - 4s - loss: 0.0215 - acc: 0.9856 - val_loss: 2.5481 - val_acc: 0.6680
0.626563028050017


In [20]:
def lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [21]:
print(lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 746s - loss: 0.6867 - acc: 0.5555 - val_loss: 0.6791 - val_acc: 0.5712
Epoch 2/10
 - 733s - loss: 0.6543 - acc: 0.6186 - val_loss: 0.6552 - val_acc: 0.6085
Epoch 3/10
 - 727s - loss: 0.6139 - acc: 0.6522 - val_loss: 0.6526 - val_acc: 0.6154
Epoch 4/10
 - 727s - loss: 0.5578 - acc: 0.7042 - val_loss: 0.6338 - val_acc: 0.6322
Epoch 5/10
 - 725s - loss: 0.4748 - acc: 0.7635 - val_loss: 0.7295 - val_acc: 0.6295
Epoch 6/10
 - 733s - loss: 0.3932 - acc: 0.8227 - val_loss: 0.7149 - val_acc: 0.6490
Epoch 7/10
 - 821s - loss: 0.3050 - acc: 0.8696 - val_loss: 0.8015 - val_acc: 0.6430
Epoch 8/10
 - 904s - loss: 0.2237 - acc: 0.9092 - val_loss: 0.8674 - val_acc: 0.6481
Epoch 9/10
 - 751s - loss: 0.1474 - acc: 0.9420 - val_loss: 1.1117 - val_acc: 0.6493
Epoch 10/10
 - 731s - loss: 0.1111 - acc: 0.9606 - val_loss: 1.2688 - val_acc: 0.6388
0.5880740233036327


In [22]:
def lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [23]:
print(lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 762s - loss: 0.6821 - acc: 0.5708 - val_loss: 0.6631 - val_acc: 0.5980
Epoch 2/10
 - 760s - loss: 0.6142 - acc: 0.6664 - val_loss: 0.6728 - val_acc: 0.6061
Epoch 3/10
 - 761s - loss: 0.4814 - acc: 0.7704 - val_loss: 0.6501 - val_acc: 0.6493
Epoch 4/10
 - 761s - loss: 0.3386 - acc: 0.8524 - val_loss: 0.7271 - val_acc: 0.6478
Epoch 5/10
 - 761s - loss: 0.2066 - acc: 0.9209 - val_loss: 0.9712 - val_acc: 0.6526
Epoch 6/10
 - 760s - loss: 0.1355 - acc: 0.9471 - val_loss: 1.0889 - val_acc: 0.6487
Epoch 7/10
 - 762s - loss: 0.0831 - acc: 0.9705 - val_loss: 1.2986 - val_acc: 0.6569
Epoch 8/10
 - 762s - loss: 0.0638 - acc: 0.9739 - val_loss: 1.4666 - val_acc: 0.6593
Epoch 9/10
 - 763s - loss: 0.0556 - acc: 0.9787 - val_loss: 1.5878 - val_acc: 0.6526
Epoch 10/10
 - 761s - loss: 0.0495 - acc: 0.9805 - val_loss: 1.6620 - val_acc: 0.6535
0.6076896903708745
